In [20]:
!wget https://uacrmq.bn.files.1drv.com/y4m2U2GY-s4-87nkKHYBUPZTU0vUfz1QAcNXAeKHjCo3YGQJOzavBfNhvzlMpIJsNnWqjEEpIL6Px6fooytvA66v48ncK2yoa4ysRvGpJckVv8lrS7sOrsSHZV3QF6kXMtKzSc0W6ACLUelLwWE6HuWIf3FFPNvLENZOJ0NfYfrJ-_cr0cwVzDQM5hJHqGHYtXt/fruits-360_dataset.zip
  
 

--2018-07-05 13:26:20--  https://uacrmq.bn.files.1drv.com/y4m2U2GY-s4-87nkKHYBUPZTU0vUfz1QAcNXAeKHjCo3YGQJOzavBfNhvzlMpIJsNnWqjEEpIL6Px6fooytvA66v48ncK2yoa4ysRvGpJckVv8lrS7sOrsSHZV3QF6kXMtKzSc0W6ACLUelLwWE6HuWIf3FFPNvLENZOJ0NfYfrJ-_cr0cwVzDQM5hJHqGHYtXt/fruits-360_dataset.zip
Resolving uacrmq.bn.files.1drv.com (uacrmq.bn.files.1drv.com)... 204.79.197.213
Connecting to uacrmq.bn.files.1drv.com (uacrmq.bn.files.1drv.com)|204.79.197.213|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 405084507 (386M) [application/zip]
Saving to: ‘fruits-360_dataset.zip’

fruits-360_dataset. 100%[===================>] 386.32M  18.2MB/s    in 22s     

2018-07-05 13:26:43 (17.7 MB/s) - ‘fruits-360_dataset.zip’ saved [405084507/405084507]



In [23]:
# import zipfile
# zip_ref = zipfile.ZipFile('fruits-360_dataset.zip', 'r')
# zip_ref.extractall('data2')
# zip_ref.close()

In [27]:
!ls

data2				   weights.03-0.937.hdf5  weights.14-0.965.hdf5
datalab				   weights.05-0.939.hdf5  weights.17-0.968.hdf5
fruits-360_dataset_2018_06_03.zip  weights.05-0.961.hdf5  weights.20-0.965.hdf5
fruits-360_dataset.zip		   weights.06-0.957.hdf5  weights.21-0.970.hdf5
weights.01-0.898.hdf5		   weights.08-0.964.hdf5  weights.30-0.971.hdf5
weights.01-0.918.hdf5		   weights.09-0.958.hdf5  weights.50-0.972.hdf5
weights.02-0.931.hdf5		   weights.10-0.962.hdf5
weights.02-0.955.hdf5		   weights.12-0.964.hdf5


In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import glob
import os
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Using TensorFlow backend.


In [0]:
#Load dữ liệu tập train và test

In [2]:
training_fruit_img = []
training_label = []
for dir_path in glob.glob("data2/fruits-360/Training/*"):
    img_label = dir_path.split("/")[-1]
    for image_path in glob.glob(os.path.join(dir_path,"*.jpg")):
        image = cv2.imread(image_path,cv2.IMREAD_COLOR)
        image = cv2.resize(image, (64, 64))
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        training_fruit_img.append(image)
        training_label.append(img_label)
training_fruit_img = np.array(training_fruit_img)
training_label = np.array(training_label)
label_to_id = {v:k for k,v in enumerate(np.unique(training_label)) }
id_to_label = {v:k for k,v in label_to_id.items()}
training_label_id = np.array([label_to_id[i] for i in training_label]) # chuyển trainlabel từ tên trái cây thành chỉ số từ 0->64
validation_fruit_img=[]
validation_label =[]
for dir_path in glob.glob("data2/fruits-360/Test/*"):
    img_label = dir_path.split("/")[-1]
    for image_path in glob.glob(os.path.join(dir_path,"*.jpg")):
        image = cv2.imread(image_path,cv2.IMREAD_COLOR)
        image = cv2.resize(image, (64, 64))
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        validation_fruit_img.append(image)
        validation_label.append(img_label)
validation_fruit_img = np.array(validation_fruit_img)
validation_label = np.array(validation_label)
validation_label_id = np.array([label_to_id[i] for i in validation_label]) # chuyển validationlabel từ tên trái cây thành chỉ số từ 0->64

In [30]:
validation_fruit_img

array([], dtype=float64)

In [4]:
#tạo cnn model

batch_size = 128
num_classes = 75
epochs = 50

# input image dimensions
img_rows, img_cols = 64, 64
input_shape = (img_rows,img_cols,3)


y_train = training_label_id
y_test = validation_label_id

x_train = training_fruit_img.astype('float32')
x_test = validation_fruit_img.astype('float32')

# chuyển các giá trị dữ liệu hình ảnh thành 0->1
x_test /= 255
x_train /= 255

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# chuyển sang onehot vector
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

#tạo checkpoint để lưu lại model tại epoch có giá trị val_acc cao nhất
checkpoint = keras.callbacks.ModelCheckpoint(filepath = "weights.{epoch:02d}-{val_acc:.3f}.hdf5", monitor='val_acc', verbose=0, save_best_only=True,
                                             save_weights_only=False, mode='max', period=1)
model = Sequential()

model.add(Conv2D(32, kernel_size= (3,3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, kernel_size = (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, kernel_size= (3,3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, kernel_size = (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(256, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
            optimizer = keras.optimizers.RMSprop(lr=0.0005, rho=0.9, epsilon=1e-08, decay=0.0),
             metrics=['accuracy'])
model.fit(x_train, y_train,
        batch_size=batch_size,
        epochs=epochs,
        verbose=1,
        validation_data=(x_test, y_test),
         callbacks = [checkpoint])
score = model.evaluate(x_test, y_test, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

from sklearn.metrics import classification_report
classes = model.predict_classes(x_test, batch_size = None)
print(classification_report(validation_label_id, classes))


x_train shape: (37836, 64, 64, 3)
37836 train samples
12709 test samples
Train on 37836 samples, validate on 12709 samples
Epoch 1/50
37836/37836 [==============================] - 30s 804us/step - loss: 1.4042 - acc: 0.6061 - val_loss: 0.4007 - val_acc: 0.8731
Epoch 2/50
12544/37836 [========>.....................] - ETA: 17s - loss: 0.2737 - acc: 0.9130

37836/37836 [==============================] - 29s 776us/step - loss: 0.2057 - acc: 0.9347 - val_loss: 0.2389 - val_acc: 0.9278
Epoch 3/50
37836/37836 [==============================] - 29s 775us/step - loss: 0.0969 - acc: 0.9685 - val_loss: 0.1886 - val_acc: 0.9469
Epoch 4/50
 6144/37836 [===>..........................] - ETA: 22s - loss: 0.0745 - acc: 0.9762

37836/37836 [==============================] - 29s 775us/step - loss: 0.0663 - acc: 0.9774 - val_loss: 0.2638 - val_acc: 0.9312
Epoch 5/50
37836/37836 [==============================] - 29s 776us/step - loss: 0.0456 - acc: 0.9831 - val_loss: 0.1611 - val_acc: 0.9507
Epoch 6/50
 4608/37836 [==>...........................] - ETA: 22s - loss: 0.0323 - acc: 0.9872

37836/37836 [==============================] - 29s 773us/step - loss: 0.0394 - acc: 0.9862 - val_loss: 0.1236 - val_acc: 0.9592
Epoch 7/50
37836/37836 [==============================] - 29s 772us/step - loss: 0.0339 - acc: 0.9877 - val_loss: 0.2644 - val_acc: 0.9318
Epoch 8/50
 4224/37836 [==>...........................] - ETA: 23s - loss: 0.0304 - acc: 0.9901

37836/37836 [==============================] - 29s 773us/step - loss: 0.0324 - acc: 0.9881 - val_loss: 0.1410 - val_acc: 0.9572
Epoch 9/50
37836/37836 [==============================] - 29s 773us/step - loss: 0.0294 - acc: 0.9893 - val_loss: 0.1220 - val_acc: 0.9632
Epoch 10/50
 4096/37836 [==>...........................] - ETA: 23s - loss: 0.0243 - acc: 0.9893

...

37836/37836 [==============================] - 29s 772us/step - loss: 0.0164 - acc: 0.9922 - val_loss: 0.1755 - val_acc: 0.9749
Epoch 43/50
37836/37836 [==============================] - 29s 771us/step - loss: 0.0201 - acc: 0.9918 - val_loss: 0.2341 - val_acc: 0.9684
Epoch 44/50
 3968/37836 [==>...........................] - ETA: 23s - loss: 0.0072 - acc: 0.9950

37836/37836 [==============================] - 29s 771us/step - loss: 0.0172 - acc: 0.9925 - val_loss: 0.2610 - val_acc: 0.9662
Epoch 45/50
37836/37836 [==============================] - 29s 770us/step - loss: 0.0175 - acc: 0.9916 - val_loss: 0.2392 - val_acc: 0.9695
Epoch 46/50
 3968/37836 [==>...........................] - ETA: 23s - loss: 0.0152 - acc: 0.9934

37836/37836 [==============================] - 29s 773us/step - loss: 0.0173 - acc: 0.9925 - val_loss: 0.2347 - val_acc: 0.9672
Epoch 47/50
37836/37836 [==============================] - 29s 772us/step - loss: 0.0179 - acc: 0.9924 - val_loss: 0.3373 - val_acc: 0.9675
Epoch 48/50
 3968/37836 [==>...........................] - ETA: 23s - loss: 0.0204 - acc: 0.9937

37836/37836 [==============================] - 29s 771us/step - loss: 0.0176 - acc: 0.9930 - val_loss: 0.3987 - val_acc: 0.9644
Epoch 49/50
37836/37836 [==============================] - 29s 771us/step - loss: 0.0186 - acc: 0.9928 - val_loss: 0.2039 - val_acc: 0.9729
Epoch 50/50
 3968/37836 [==>...........................] - ETA: 23s - loss: 0.0159 - acc: 0.9932

37836/37836 [==============================] - 29s 772us/step - loss: 0.0203 - acc: 0.9921 - val_loss: 0.2304 - val_acc: 0.9670
Test loss: 0.23039191525035965
Test accuracy: 0.967031237705563
             precision    recall  f1-score   support

          0       0.77      0.94      0.85       164
          1       1.00      0.99      1.00       164
          2       0.98      1.00      0.99       164
          3       0.80      1.00      0.89       161
          4       1.00      0.97      0.98       164
          5       1.00      0.74      0.85       164
          6       1.00      0.91      0.95       164
          7       1.00      1.00      1.00       144
          8       1.00      1.00      1.00       166
          9       0.92      1.00      0.96       164
         10       0.86      1.00      0.93       164
         11       1.00      1.00      1.00       143
         12       1.00      1.00      1.00       166
         13       1.00      0.99      1.00       166
         14 

In [0]:
#tại epochs = 14 đạt giá trị val_acc = 0.976 cao nhất: load model tại epoch = 42:



In [14]:
!ls




data2				   weights.04-0.927.hdf5  weights.10-0.955.hdf5
datalab				   weights.04-0.970.hdf5  weights.10-0.962.hdf5
fruits-360_dataset_2018_06_03.zip  weights.05-0.939.hdf5  weights.11-0.958.hdf5
fruits-360_dataset.zip		   weights.05-0.940.hdf5  weights.11-0.970.hdf5
weights.01-0.785.hdf5		   weights.05-0.951.hdf5  weights.12-0.958.hdf5
weights.01-0.862.hdf5		   weights.05-0.952.hdf5  weights.12-0.964.hdf5
weights.01-0.873.hdf5		   weights.05-0.959.hdf5  weights.13-0.966.hdf5
weights.01-0.898.hdf5		   weights.05-0.961.hdf5  weights.14-0.963.hdf5
weights.01-0.911.hdf5		   weights.05-0.970.hdf5  weights.14-0.965.hdf5
weights.01-0.918.hdf5		   weights.06-0.953.hdf5  weights.14-0.968.hdf5
weights.01-0.937.hdf5		   weights.06-0.954.hdf5  weights.17-0.966.hdf5
weights.02-0.889.hdf5		   weights.06-0.957.hdf5  weights.17-0.968.hdf5
weights.02-0.928.hdf5		   weights.06-0.959.hdf5  weights.17-0.969.hdf5
weights.02-0.931.hdf5		   weights.06-0.961.hdf5  weights.20-0.965.hdf5
weights.02-0.936.hdf5	

In [22]:
!rm -r data2


In [6]:
modelbest = keras.models.load_model('weights.42-0.975.hdf5')
from sklearn.metrics import classification_report
classes = modelbest.predict_classes(x_test, batch_size = None)
print(classification_report(validation_label_id, classes))

             precision    recall  f1-score   support

          0       0.96      0.78      0.86       164
          1       1.00      0.93      0.97       164
          2       0.83      1.00      0.91       164
          3       0.77      1.00      0.87       161
          4       1.00      0.91      0.95       164
          5       1.00      0.92      0.96       164
          6       1.00      1.00      1.00       164
          7       1.00      1.00      1.00       144
          8       1.00      1.00      1.00       166
          9       1.00      1.00      1.00       164
         10       1.00      1.00      1.00       164
         11       1.00      1.00      1.00       143
         12       0.98      1.00      0.99       166
         13       0.99      1.00      1.00       166
         14       0.93      0.94      0.93       166
         15       1.00      0.99      1.00       166
         16       1.00      1.00      1.00       164
         17       1.00      0.99      1.00   

In [7]:
modelbest.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 62, 62, 32)        896       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 60, 60, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 30, 30, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 28, 28, 32)        18464     
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 26, 26, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 13, 13, 64)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 13, 13, 64)        0         
__________

In [19]:
# sử dụng randomforest để phân loại

rf = RandomForestClassifier(n_estimators=100)
rf.fit(training_fruit_img.reshape(training_fruit_img.shape[0],64*64*3), training_label_id)
pred = rf.predict(validation_fruit_img.reshape(validation_fruit_img.shape[0],64*64*3))


from sklearn.metrics import classification_report
print(classification_report(y_test, pred))
print(np.sum(y_test==pred))

             precision    recall  f1-score   support

          0       0.76      0.71      0.74       164
          1       0.77      0.94      0.85       164
          2       0.94      1.00      0.97       164
          3       1.00      1.00      1.00       161
          4       1.00      1.00      1.00       164
          5       1.00      1.00      1.00       164
          6       0.93      1.00      0.96       164
          7       1.00      1.00      1.00       144
          8       1.00      1.00      1.00       166
          9       0.99      0.76      0.86       164
         10       1.00      1.00      1.00       164
         11       1.00      0.88      0.94       143
         12       1.00      1.00      1.00       166
         13       0.99      0.80      0.88       166
         14       0.80      0.84      0.82       166
         15       0.97      0.90      0.94       166
         16       0.99      1.00      1.00       164
         17       0.97      0.99      0.98   

In [31]:
#model dùng randomforest cho kết quả khá giống với cnn mặc dù accurancy thấp hơn.

In [17]:
from google.colab import files
files.download('weights.42-0.975.hdf5')  # from colab to browser download


KeyboardInterrupt: ignored

apple.jpg			     lemon1.jpg  thahlongtest.jpg
data2				     lemon.jpg	 tmp
datalab				     logs	 vai.jpg
fruits-360_dataset_2018_06_03.zip    logs2
fruits-360_dataset_2018_06_03.zip.1  logs2.zip
